In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

#### 1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).  

In [2]:
def calc_logloss(y, p):
    p = np.clip(p, 1e-5, 1 - 1e-5)
    err = np.mean(- y * np.log(p) - (1.0 - y) * np.log(1.0 - p))
    return err

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [4]:
def standardization(X):
    S = (X - X.mean(axis=0)) / X.std(ddof=1, axis=0)
    return S

In [5]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [21, 25, 27, 20, 25, 18, 35, 60, 20, 30]],  # возраст репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [6]:
X_st = X.copy()
X_st[:, 1:] = standardization(X[:, 1:])
X_st

array([[ 1.        , -0.54210474, -0.92932038, -0.57485743],
       [ 1.        , -0.54210474, -0.53802759, -0.25099409],
       [ 1.        , -0.20328928, -0.44020439, -0.08906242],
       [ 1.        , -0.54210474, -0.73367398, -0.65582327],
       [ 1.        ,  0.13552619,  0.92932038, -0.25099409],
       [ 1.        , -0.88092021, -0.34238119, -0.81775494],
       [ 1.        ,  0.81315711,  1.02714357,  0.55866427],
       [ 1.        ,  2.50723443,  2.00537555,  2.58281015],
       [ 1.        , -0.54210474, -1.02714357, -0.65582327],
       [ 1.        , -0.20328928,  0.0489116 ,  0.15383509]])

#### 2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.

In [7]:
def eval_LR_model(X, y, iterations, eta=1e-4):
    np.random.seed(66)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, pred)
        w -= eta * (1/n * np.dot((pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [8]:
w = eval_LR_model(X_st, y, 3000, 1e-3)

300 [ 1.34030982 -1.0019899  -0.52123324 -0.68507542] 1.2737138773105383
600 [ 1.26663347 -0.92150377 -0.42933575 -0.61558661] 1.1897570767279477
900 [ 1.19486741 -0.84439278 -0.34115474 -0.54902807] 1.1120533254149838
1200 [ 1.12525959 -0.77133094 -0.25731062 -0.48609895] 1.0414399318811265
1500 [ 1.0580051  -0.70306513 -0.17846764 -0.42758258] 0.9786503191074895
1800 [ 0.99324179 -0.64032906 -0.10525659 -0.37425798] 0.9241547776790007
2100 [ 0.93106358 -0.58371735 -0.03816741 -0.32676907] 0.8780092668733698
2400 [ 0.87154324 -0.53356341  0.02255083 -0.28549683] 0.8397934351125824
2700 [ 0.81474873 -0.48987738  0.07693695 -0.25049245] 0.8086846629314678
3000 [ 0.76074374 -0.45236904  0.12529265 -0.22149823] 0.7836322747815148


#### 3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`.

In [9]:
def calc_pred_proba(X, w):
    return sigmoid(np.dot(X, w))    

In [10]:
calc_pred_proba(X_st, w)

array([0.73436208, 0.72990862, 0.6936583 , 0.7425596 , 0.70505099,
       0.78541026, 0.59817028, 0.33308152, 0.7354681 , 0.69524573])

#### 4. Создайте функцию `calc_pred`, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, а также порог вероятности.

In [11]:
def calc_pred(X, w, treshold=.5):
    p = calc_pred_proba(X, w)
    y_pred = np.where(p >= treshold, 1, 0)
    return y_pred

In [12]:
y_pred = calc_pred(X_st, w, treshold=.55)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [13]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

#### 5. (\*) Напишите функции для расчета accuracy, матрицы ошибок, precision и recall, а также F1-score.

In [14]:
def confusion_matrix(y_pred, y):
    tp = np.sum(((y_pred == 1) & (y == 1)) * 1)
    tn = np.sum(((y_pred == 0) & (y == 0)) * 1)
    fp = np.sum(((y_pred == 1) & (y == 0)) * 1)
    fn = np.sum(((y_pred == 0) & (y == 1)) * 1)
    matrix = np.array([
        [tp, fn],
        [fp, tn]
    ])
    return matrix

In [15]:
def accuracy(y_pred, y):
    return np.sum((y_pred == y) * 1)/ y.shape[0]

In [16]:
def precision(y_pred, y):
    cf = confusion_matrix(y_pred, y)
    return cf[0, 0] / (cf[0, 0] + cf[0, 1])

In [17]:
def recall(y_pred, y):
    cf = confusion_matrix(y_pred, y)
    return cf[0, 0] / (cf[0, 0] + cf[1, 0])

In [18]:
def f1_score(y_pred, y):
    return (2 * precision(y_pred, y) * recall(y_pred, y)) / (precision(y_pred, y) + recall(y_pred, y))

In [19]:
def show_errors(y_pred, y):
    print(f'Accuracy  = {accuracy(y_pred, y):.3f}\nPrecision = {precision(y_pred, y):.3f}')
    print(f'Recall    = {recall(y_pred, y):.3f}\nF1-score  = {f1_score(y_pred, y):.3f}')
    print(f'\nConfusion matrix = \n{confusion_matrix(y_pred, y)}')

In [20]:
show_errors(y_pred, y)

Accuracy  = 0.600
Precision = 1.000
Recall    = 0.556
F1-score  = 0.714

Confusion matrix = 
[[5 0]
 [4 1]]


Модель плохо определяет `класс 0`, причем эту проблему не получится решить изменением порога определения `класса 1`.
Похоже, что для адекватной оценки вероятности определения `класса 1` требуется больше данных.